# Modulos, Clases y funciones Externas

In [1]:
# Librerías estándar de Python
import datetime  # Manejo de fechas y horas
import pandas as pd  # Manipulación y análisis de datos en DataFrames
import numpy as np  # Operaciones numéricas y manejo eficiente de arrays
import sys  # Acceso a variables y funciones del sistema
import re  # Expresiones regulares para procesamiento de texto
import os  # Operaciones del sistema de archivos

# Añadir la carpeta raíz del proyecto al sys.path para importar módulos personalizados
sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
#sys.path.append(os.path.abspath("D:\Proyectos Python\capresoca-data-automation"))  # Ruta alternativa (comentada)

# Importar función y clase personalizada del proyecto
from src.file_loader import cargar_maestros_ADRES  # Función para cargar archivos maestros ADRES
from src.data_cleaning import BduaReportProcessor      # Clase para limpiar y normalizar población Maestro ADRES
from src.data_cleaning import DataCleaner # Clase para limpiar y normalizar DataFrames de Pandas

# Rutas y varaibles

In [2]:
R_Pila_Validada = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\08_Agosto\20\Dataframe Pila 20-08-2025.xlsx"
R_Maestro__EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0019082025.TXT"
R_Maestro__EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-02\EPS025MS0019082025.TXT"

fecha_archivo = "20-08-2025"

S_Excel = fr"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\08_Agosto\20\S1 Estructura {fecha_archivo}.xlsx"

# Crear el objeto de fecha
fecha_Minima = datetime.datetime(2025, 6, 1)
fecha_reporte = datetime.datetime(2025, 8, 20)
fecha_proxi_reporte = datetime.datetime(2025, 9, 2)

# Cargue Dataframes

In [3]:
# Cargar y combinar los maestros
maestro_ADRES = cargar_maestros_ADRES(R_Maestro__EPS025, R_Maestro__EPSC25)
Pila_Validada = pd.read_excel(R_Pila_Validada, sheet_name="Sheet1", header=0, dtype=str)

# Listado censal o Sisben ADRES

In [4]:
# Duplicar la columna "MARCASISBENIV+MARCASISBENIII_2" y nombrarla "MARCASISBENIV+MARCASISBENIII"
maestro_ADRES["MARCASISBENIV+MARCASISBENIII_2"] = \
    maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]

In [5]:
# 1. Instanciar el procesador: Se crea un objeto pasando el DataFrame.
#    La jerarquía de población ya está definida por defecto dentro de la clase.
processor = BduaReportProcessor(df=maestro_ADRES)

# 2. Ejecutar la limpieza y asignarla de vuelta.
#    El método retorna un DataFrame completamente nuevo con la columna actualizada.
maestro_ADRES = processor.prioritize_population_markers(
    col_name="MARCASISBENIV+MARCASISBENIII"
)

# ¡Listo! 'maestro_ADRES' ahora contiene los datos limpios.

# Contrucción S1 movilidad

## Pila

In [6]:
# Definir las columnas del nuevo DataFrame
new_columns = [
    "ENT_ID", "TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "AFL_PRIMER_APELLIDO", "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE", "AFL_SEGUNDO_NOMBRE", "AFL_FECHA_NACIMIENTO", "TPS_GNR_ID", "TPS_IDN_ID_2",
    "HST_IDN_NUMERO_IDENTIFICACION_2", "AFL_PRIMER_APELLIDO_2", "AFL_SEGUNDO_APELLIDO_2", "AFL_PRIMER_NOMBRE_2",
    "AFL_SEGUNDO_NOMBRE_2", "AFL_FECHA_NACIMIENTO_2", "TPS_GNR_ID_2", "DPR_ID", "MNC_ID", "ZNS_ID",
    "FECHA_AFILIACION_MOVILIDAD", "TPS_GRP_PBL_ID", "TPS_NVL_SSB_ID", "TIPO_TRASLADO", "CND_AFL_SBS_METODOLOGIA",
    "CND_AFL_SBS_SUBGRUPO_SIV", "CON_DISCAPACIDAD", "TPS_IDN_CF_ID", "HST_IDN_NUMERO_CF_IDENTIFICACION",
    "TPS_PRN_ID", "TPS_AFL_ID", "TPS_ETN_ID", "NOM_RESGUARDO_INDIGENA",
    "PAIS_NACIMIENTO", "LUGAR_NACIMIENTO", "NACIONALIDAD", "SEXO_IDENTIFICACION", "TIPO_DISCAPACIDAD"
]

# Filtrar registros donde 'Movilidad' contiene 'S1'
mask = Pila_Validada["Movilidad"].str.contains("S1", na=False)
filtered = Pila_Validada.loc[mask]

# Crear el nuevo DataFrame S1 con valores vacíos
S1 = pd.DataFrame('', index=filtered.index, columns=new_columns)

# Asignar los valores requeridos
S1.loc[:, "TPS_IDN_ID"] = filtered["Tipo Documento Cotizante"]
S1.loc[:, "HST_IDN_NUMERO_IDENTIFICACION"] = filtered["N° Identificación Cotizante"]
S1.loc[:, "FECHA_AFILIACION_MOVILIDAD"] = filtered["Fecha envio"]
S1.loc[:, "CND_AFL_SBS_SUBGRUPO_SIV"] = filtered["Población_2"]

# Mostrar cantidad de registros, columnas y porcentaje de vacíos por columna
print(f"Registros: {S1.shape[0]}, Columnas: {S1.shape[1]}")
print("Porcentaje de vacíos por columna:")
print(S1.isin(['', None, np.nan]).mean() * 100)

Registros: 788, Columnas: 38
Porcentaje de vacíos por columna:
ENT_ID                              100.0
TPS_IDN_ID                            0.0
HST_IDN_NUMERO_IDENTIFICACION         0.0
AFL_PRIMER_APELLIDO                 100.0
AFL_SEGUNDO_APELLIDO                100.0
AFL_PRIMER_NOMBRE                   100.0
AFL_SEGUNDO_NOMBRE                  100.0
AFL_FECHA_NACIMIENTO                100.0
TPS_GNR_ID                          100.0
TPS_IDN_ID_2                        100.0
HST_IDN_NUMERO_IDENTIFICACION_2     100.0
AFL_PRIMER_APELLIDO_2               100.0
AFL_SEGUNDO_APELLIDO_2              100.0
AFL_PRIMER_NOMBRE_2                 100.0
AFL_SEGUNDO_NOMBRE_2                100.0
AFL_FECHA_NACIMIENTO_2              100.0
TPS_GNR_ID_2                        100.0
DPR_ID                              100.0
MNC_ID                              100.0
ZNS_ID                              100.0
FECHA_AFILIACION_MOVILIDAD            0.0
TPS_GRP_PBL_ID                      100.0
TPS_NVL_SSB_I

In [7]:
S1

,ENT_ID,TPS_IDN_ID,HST_IDN_NUMERO_IDENTIFICACION,AFL_PRIMER_APELLIDO,AFL_SEGUNDO_APELLIDO,AFL_PRIMER_NOMBRE,AFL_SEGUNDO_NOMBRE,AFL_FECHA_NACIMIENTO,TPS_GNR_ID,TPS_IDN_ID_2,...,HST_IDN_NUMERO_CF_IDENTIFICACION,TPS_PRN_ID,TPS_AFL_ID,TPS_ETN_ID,NOM_RESGUARDO_INDIGENA,PAIS_NACIMIENTO,LUGAR_NACIMIENTO,NACIONALIDAD,SEXO_IDENTIFICACION,TIPO_DISCAPACIDAD
0,,CC,1000002981,,,,,,,,...,,,,,,,,,,
14,,CC,1000378486,,,,,,,,...,,,,,,,,,,
19,,CC,1000474391,,,,,,,,...,,,,,,,,,,
34,,CC,1000805521,,,,,,,,...,,,,,,,,,,
50,,CC,1001097279,,,,,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17311,,PT,5440800,,,,,,,,...,,,,,,,,,,
17364,,PT,6021201,,,,,,,,...,,,,,,,,,,
17398,,PT,6325913,,,,,,,,...,,,,,,,,,,
17455,,PT,6880548,,,,,,,,...,,,,,,,,,,


In [8]:
S1["Where"] = "Pila"

## Beneficairios

In [9]:
import pandas as pd

# ----------------------------------------------------
# Debug: mostrar nombres de columnas antes de cualquier operación
# ----------------------------------------------------
print(">> S1 columns antes de strip():")
print([repr(c) for c in S1.columns.tolist()])
print(">> maestro_ADRES columns antes de strip():")
print([repr(c) for c in maestro_ADRES.columns.tolist()])

# ----------------------------------------------------
# Limpiar espacios invisibles en los nombres de columna
# ----------------------------------------------------
S1.columns            = S1.columns.str.strip()
maestro_ADRES.columns = maestro_ADRES.columns.str.strip()

# ----------------------------------------------------
# Debug: volver a mostrar nombres de columnas después de strip()
# ----------------------------------------------------
print(">> S1 columns después de strip():")
print([repr(c) for c in S1.columns.tolist()])
print(">> maestro_ADRES columns después de strip():")
print([repr(c) for c in maestro_ADRES.columns.tolist()])

# ----------------------------------------------------
# 0) Cantidad de registros y columnas en S1
# ----------------------------------------------------
print(f"Registros: {S1.shape[0]}, Columnas: {S1.shape[1]}")

# ----------------------------------------------------
# 1) Preparar sólo las claves de S1 (cabezas de familia),
#    incluyendo FECHA_AFILIACION_MOVILIDAD para luego propagarla
# ----------------------------------------------------
s1_keys = (
    S1[["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "FECHA_AFILIACION_MOVILIDAD"]]
    .drop_duplicates(subset=["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])
)

# ----------------------------------------------------
# 2) Emparejar S1 (cabezas) con maestro_ADRES (beneficiarios)
#    - Usamos sufijos para distinguir columnas duplicadas
# ----------------------------------------------------
beneficiarios = (
    maestro_ADRES
    .merge(
        s1_keys,
        left_on=["TPS_IDN_ID_CF", "HST_IDN_NUMERO_IDENTIFICACION_CF"],
        right_on=["TPS_IDN_ID",    "HST_IDN_NUMERO_IDENTIFICACION"],
        how="inner",
        suffixes=("_master", "_s1")
    )
)

# Debug: ver columnas que resultaron de este merge
print(">> beneficiarios columns:")
print(beneficiarios.columns.tolist())

# ----------------------------------------------------
# 3) Construir los nuevos registros con las columnas solicitadas,
#    propagando FECHA_AFILIACION_MOVILIDAD desde la familia cabeza
# ----------------------------------------------------
nuevos = pd.DataFrame({
    # ID del beneficiario (de maestro_ADRES con sufijo _master)
    "TPS_IDN_ID":                        beneficiarios["TPS_IDN_ID_master"],
    "HST_IDN_NUMERO_IDENTIFICACION":     beneficiarios["HST_IDN_NUMERO_IDENTIFICACION_master"],
    # Subgrupo según MARCASISBENIV+MARCASISBENIII
    "CND_AFL_SBS_SUBGRUPO_SIV":          beneficiarios["MARCASISBENIV+MARCASISBENIII"],
    # IDs de la cabeza de familia
    "TPS_IDN_CF_ID":                     beneficiarios["TPS_IDN_ID_CF"],
    "HST_IDN_NUMERO_CF_IDENTIFICACION":  beneficiarios["HST_IDN_NUMERO_IDENTIFICACION_CF"],
    # TPS_PRN_ID del beneficiario
    "TPS_PRN_ID":                        beneficiarios["TPS_PRN_ID"],
    # Marcamos estos nuevos registros con la etiqueta fija
    "Where":                             "Beneficiarios Pila",
    # Propagamos la misma fecha de movilidad que tenía la cabeza de familia
    "FECHA_AFILIACION_MOVILIDAD":        beneficiarios["FECHA_AFILIACION_MOVILIDAD"],
})

# ----------------------------------------------------
# 4) Añadir los nuevos registros al final de S1
# ----------------------------------------------------
S1 = pd.concat([S1, nuevos], ignore_index=True)

# ----------------------------------------------------
# 5) Mostrar resumen final
# ----------------------------------------------------
print(f"Registros después del concat: {S1.shape[0]}")
print(f"Columnas después del concat:  {S1.shape[1]}")


>> S1 columns antes de strip():
["'ENT_ID'", "'TPS_IDN_ID'", "'HST_IDN_NUMERO_IDENTIFICACION'", "'AFL_PRIMER_APELLIDO'", "'AFL_SEGUNDO_APELLIDO'", "'AFL_PRIMER_NOMBRE'", "'AFL_SEGUNDO_NOMBRE'", "'AFL_FECHA_NACIMIENTO'", "'TPS_GNR_ID'", "'TPS_IDN_ID_2'", "'HST_IDN_NUMERO_IDENTIFICACION_2'", "'AFL_PRIMER_APELLIDO_2'", "'AFL_SEGUNDO_APELLIDO_2'", "'AFL_PRIMER_NOMBRE_2'", "'AFL_SEGUNDO_NOMBRE_2'", "'AFL_FECHA_NACIMIENTO_2'", "'TPS_GNR_ID_2'", "'DPR_ID'", "'MNC_ID'", "'ZNS_ID'", "'FECHA_AFILIACION_MOVILIDAD'", "'TPS_GRP_PBL_ID'", "'TPS_NVL_SSB_ID'", "'TIPO_TRASLADO'", "'CND_AFL_SBS_METODOLOGIA'", "'CND_AFL_SBS_SUBGRUPO_SIV'", "'CON_DISCAPACIDAD'", "'TPS_IDN_CF_ID'", "'HST_IDN_NUMERO_CF_IDENTIFICACION'", "'TPS_PRN_ID'", "'TPS_AFL_ID'", "'TPS_ETN_ID'", "'NOM_RESGUARDO_INDIGENA'", "'PAIS_NACIMIENTO'", "'LUGAR_NACIMIENTO'", "'NACIONALIDAD'", "'SEXO_IDENTIFICACION'", "'TIPO_DISCAPACIDAD'", "'Where'"]
>> maestro_ADRES columns antes de strip():
["'AFL_ID'", "'ENT_ID'", "'TPS_IDN_ID_CF'", "'HST_IDN

### Fechas de Beneficairios

## Sisben Beneficiarios

In [10]:
# Realizar el mapeo de los valores de maestro_ADRES["MARCASISBENIV+MARCASISBENIII"]
# a S1["CND_AFL_SBS_SUBGRUPO_SIV"] solo donde S1["Where"] == "Beneficiarios Pila"

# Crear un diccionario para búsqueda rápida por ID
maestro_map = maestro_ADRES.set_index(["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"])["MARCASISBENIV+MARCASISBENIII"]

# Actualizar S1 usando .apply para mantener el nombre de las columnas
def actualizar_subgrupo(row):
    if row.get("Where") == "Beneficiarios Pila":
        key = (row["TPS_IDN_ID"], row["HST_IDN_NUMERO_IDENTIFICACION"])
        valor = maestro_map.get(key, row["CND_AFL_SBS_SUBGRUPO_SIV"])
        return valor
    return row["CND_AFL_SBS_SUBGRUPO_SIV"]

S1["CND_AFL_SBS_SUBGRUPO_SIV"] = S1.apply(actualizar_subgrupo, axis=1)

## Datos del Maestro al S1

In [11]:
import pandas as pd

# 1) Defino cómo se llaman en maestro → cómo quiero que queden en S1
column_mapping = {
    "AFL_PAIS_NACIMIENTO":     "PAIS_NACIMIENTO",
    "AFL_MUNICIPIO_NACIMIENTO":"LUGAR_NACIMIENTO",
    "AFL_NACIONALIDAD":        "NACIONALIDAD",
    "AFL_SEXO_IDENTIFICACION": "SEXO_IDENTIFICACION",
    "AFL_DISCAPACIDAD":        "TIPO_DISCAPACIDAD",
    "DPR_ID":                  "DPR_ID",
    "MNC_ID":                  "MNC_ID",
    "ZNS_ID":                  "ZNS_ID",
    "TPS_AFL_ID":              "TPS_AFL_ID",
    # (y las que ya tenías antes…)
    "AFL_PRIMER_APELLIDO":    "AFL_PRIMER_APELLIDO",
    "AFL_SEGUNDO_APELLIDO":   "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE":      "AFL_PRIMER_NOMBRE",
    "AFL_SEGUNDO_NOMBRE":     "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO":   "AFL_FECHA_NACIMIENTO",
    "TPS_GNR_ID":             "TPS_GNR_ID",
}

join_keys = ["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"]

# 2) Preparo el maestro: me quedo solo con las columnas que necesito y
#    las renombro para que coincidan con S1:
maestro_subset = (
    maestro_ADRES[list(column_mapping.keys()) + join_keys]
    .rename(columns=column_mapping)
)

# 3) Hago el merge contra S1; pandas añadirá un sufijo solo a las columnas
#    del maestro (porque ya existen en S1):
S1_merged = S1.merge(
    maestro_subset,
    on=join_keys,
    how="left",
    suffixes=("", "_maestro")
)

# 4) Ahora, para cada columna de destino 'col', relleno S1[col] solo donde
#    esté vacío usando la versión '_maestro', y luego elimino esa columna:
for col in column_mapping.values():
    maestro_col = f"{col}_maestro"
    if maestro_col in S1_merged:
        S1_merged[col] = S1_merged[col].replace("", pd.NA) \
                                       .fillna(S1_merged[maestro_col]) \
                                       .fillna("")  # opcional volver a cadena vacía
        S1_merged.drop(columns=maestro_col, inplace=True)

# 5) S1_merged es tu S1 final, con el mismo esquema de columnas y
#    solo rellenando lo que estaba vacío:
S1 = S1_merged

In [12]:
# Asignar los valores de las columnas base a las columnas "_2" en S1
cols_base = [
    "TPS_IDN_ID",
    "HST_IDN_NUMERO_IDENTIFICACION",
    "AFL_PRIMER_APELLIDO",
    "AFL_SEGUNDO_APELLIDO",
    "AFL_PRIMER_NOMBRE",
    "AFL_SEGUNDO_NOMBRE",
    "AFL_FECHA_NACIMIENTO",
    "TPS_GNR_ID"
]
cols_2 = [
    "TPS_IDN_ID_2",
    "HST_IDN_NUMERO_IDENTIFICACION_2",
    "AFL_PRIMER_APELLIDO_2",
    "AFL_SEGUNDO_APELLIDO_2",
    "AFL_PRIMER_NOMBRE_2",
    "AFL_SEGUNDO_NOMBRE_2",
    "AFL_FECHA_NACIMIENTO_2",
    "TPS_GNR_ID_2"
]

for base, col2 in zip(cols_base, cols_2):
    S1[col2] = S1[base]

## Sisben S1 
1. Tipo de población "TPS_GRP_PBL_ID"
2. Nivel de sisben "TPS_NVL_SSB_ID"
3. Metodologia del sisben "CND_AFL_SBS_METODOLOGIA"
4. Grupo sisben "CND_AFL_SBS_SUBGRUPO_SIV"

In [13]:
import re

def extract_nvl_ssb(subgrupo):
    if pd.isna(subgrupo):
        return ""
    lc_match = re.match(r"LC\((\d+)\)", subgrupo)
    if lc_match:
        return lc_match.group(1)
    siv_match = re.match(r"SIV\([A-Z0-9]+\)", subgrupo)
    if siv_match:
        return "5"
    return ""

S1["TPS_GRP_PBL_ID"] = S1["CND_AFL_SBS_SUBGRUPO_SIV"].apply(extract_nvl_ssb)

In [14]:
# Asignación condicional según las reglas descritas
def assign_values(row):
    grp_pbl_id = row["TPS_GRP_PBL_ID"]
    subgrupo = row["CND_AFL_SBS_SUBGRUPO_SIV"]
    if grp_pbl_id != "5":
        row["TPS_NVL_SSB_ID"] = "N"
        row["CND_AFL_SBS_METODOLOGIA"] = "3"
        row["CND_AFL_SBS_SUBGRUPO_SIV"] = ""
    else:
        # Extraer la letra y número de SIV(xxx)
        match = re.match(r"SIV\(([A-Z])(\d{2})\)", subgrupo)
        if match:
            letra = match.group(1)
            numero = match.group(2)
            if letra in ["A", "B"]:
                row["TPS_NVL_SSB_ID"] = "1"
                row["CND_AFL_SBS_METODOLOGIA"] = "2"
            elif letra == "C":
                row["TPS_NVL_SSB_ID"] = "2"
                row["CND_AFL_SBS_METODOLOGIA"] = "2"
            row["CND_AFL_SBS_SUBGRUPO_SIV"] = f"{letra}{numero}"
        else:
            # Si no coincide con el patrón, dejar vacío
            row["TPS_NVL_SSB_ID"] = ""
            row["CND_AFL_SBS_METODOLOGIA"] = ""
            row["CND_AFL_SBS_SUBGRUPO_SIV"] = ""
    return row

S1 = S1.apply(assign_values, axis=1)

# Tipo y Fechas de movilidad
1. Se asigna el tipo de movilidad, acorde al corte del proceso a reportar "TIPO_TRASLADO"
2. se valida y se corigue la fecha de ser mayor a la de la fecha del reporte "FECHA_AFILIACION_MOVILIDAD"  y menor a la fecha del sigueinte reporte.

In [15]:
S1["FECHA_AFILIACION_MOVILIDAD"] 

0      2025-08-21 00:00:00
1      2025-07-31 00:00:00
2      2025-07-31 00:00:00
3      2025-09-01 00:00:00
4      2025-08-20 00:00:00
              ...         
953    2025-08-09 00:00:00
954    2025-08-02 00:00:00
955    2025-08-21 00:00:00
956    2025-08-31 00:00:00
957    2025-08-16 00:00:00
Name: FECHA_AFILIACION_MOVILIDAD, Length: 958, dtype: object

In [16]:
print(S1["FECHA_AFILIACION_MOVILIDAD"].head())
print(S1["FECHA_AFILIACION_MOVILIDAD"].dtype)

0    2025-08-21 00:00:00
1    2025-07-31 00:00:00
2    2025-07-31 00:00:00
3    2025-09-01 00:00:00
4    2025-08-20 00:00:00
Name: FECHA_AFILIACION_MOVILIDAD, dtype: object
object


In [17]:
import pandas as pd

# 1. Asignar valor 3 por defecto
S1["TIPO_TRASLADO"] = "3"

# 2. Convertir la columna de fechas a datetime si aún no lo es
if not pd.api.types.is_datetime64_any_dtype(S1["FECHA_AFILIACION_MOVILIDAD"]):
    try:
        S1["FECHA_AFILIACION_MOVILIDAD"] = pd.to_datetime(
            S1["FECHA_AFILIACION_MOVILIDAD"], errors="raise", format="%Y-%m-%d %H:%M:%S"
        )
    except Exception:
        # Si no tiene hora o viene en otro formato válido ISO
        S1["FECHA_AFILIACION_MOVILIDAD"] = pd.to_datetime(
            S1["FECHA_AFILIACION_MOVILIDAD"], errors="coerce", infer_datetime_format=True
        )

# Validar que todas las fechas se hayan convertido correctamente
errores_fecha = S1["FECHA_AFILIACION_MOVILIDAD"].isna().sum()
if errores_fecha > 0:
    print(f"❌ {errores_fecha} registros con fechas inválidas después de la conversión.")
    print(S1[S1["FECHA_AFILIACION_MOVILIDAD"].isna()].head())
else:
    print("✅ Todas las fechas fueron convertidas correctamente a datetime.")

# 3. Instanciar el limpiador (si aplica lógica adicional)
cleaner = DataCleaner(df=S1)
S1 = cleaner.process_date_column(column_name="FECHA_AFILIACION_MOVILIDAD")

# 4. Inicializar columna de validación
S1["Validación fecha"] = ""

# 5. Lógica de validación de fecha por registro
def validar_fecha(row):
    fecha = row["FECHA_AFILIACION_MOVILIDAD"]
    if pd.isnull(fecha):
        return row

    if fecha < fecha_Minima:
        row["FECHA_AFILIACION_MOVILIDAD"] = fecha_Minima
        row["TIPO_TRASLADO"] = "3"
        row["Validación fecha"] = ""
    elif fecha > fecha_reporte:
        if fecha < fecha_proxi_reporte:
            nueva_fecha = fecha - pd.DateOffset(months=1)
            row["FECHA_AFILIACION_MOVILIDAD"] = nueva_fecha
            row["TIPO_TRASLADO"] = "4"
            row["Validación fecha"] = ""
        else:
            row["TIPO_TRASLADO"] = ""
            row["Validación fecha"] = "proceso para proximos reportes"
    else:
        row["TIPO_TRASLADO"] = "3"
        row["Validación fecha"] = ""
    return row

# 6. Aplicar la función fila por fila
S1 = S1.apply(validar_fecha, axis=1)

# 7. Convertir a formato string DD/MM/YYYY para exportar
S1["FECHA_AFILIACION_MOVILIDAD"] = S1["FECHA_AFILIACION_MOVILIDAD"].apply(
    lambda x: x.strftime("%d/%m/%Y") if pd.notnull(x) else ""
)


✅ Todas las fechas fueron convertidas correctamente a datetime.
INFO: Procesando la columna de fecha 'FECHA_AFILIACION_MOVILIDAD'...
INFO: La columna ya es datetime. No se requiere conversión.


In [18]:
S1["FECHA_AFILIACION_MOVILIDAD"] 

0      21/07/2025
1      31/07/2025
2      31/07/2025
3      01/08/2025
4      20/08/2025
          ...    
953    09/08/2025
954    02/08/2025
955    21/07/2025
956    31/07/2025
957    16/08/2025
Name: FECHA_AFILIACION_MOVILIDAD, Length: 958, dtype: object

# Valores Absolutos
Valores que debe tener la estructura de manera automatica

In [19]:
S1["ENT_ID"] = "EPS025"

## Municipio Nacimiento

In [20]:
# Asignar código de municipio de nacimiento en LUGAR_NACIMIENTO si está vacío y PAIS_NACIMIENTO es "COL"
mask = (S1["LUGAR_NACIMIENTO"].isin(["", None, pd.NA])) & (S1["PAIS_NACIMIENTO"] == "COL")

def municipio_codigo(row):
    if mask.loc[row.name]:
        dpr = str(row["DPR_ID"]).zfill(2)
        mnc = str(row["MNC_ID"]).zfill(3)
        return dpr + mnc
    return row["LUGAR_NACIMIENTO"]

S1["LUGAR_NACIMIENTO"] = S1.apply(municipio_codigo, axis=1)

# Guardamos Dataframes

In [21]:

with pd.ExcelWriter(S_Excel, engine="openpyxl") as writer:
    S1.to_excel(writer, sheet_name="S1", index=False)
    #maestro_ADRES.to_excel(writer, sheet_name="maestro_ADRES", index=False)

print("Archivo Excel guardado en:", S_Excel)

Archivo Excel guardado en: C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\Traslados\Procesos BDUA\2025\08_Agosto\20\S1 Estructura 20-08-2025.xlsx


In [22]:
maestro_ADRES.columns

Index(['AFL_ID', 'ENT_ID', 'TPS_IDN_ID_CF', 'HST_IDN_NUMERO_IDENTIFICACION_CF',
       'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_PRIMER_APELLIDO',
       'AFL_SEGUNDO_APELLIDO', 'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
       'AFL_FECHA_NACIMIENTO', 'TPS_GNR_ID', 'AFL_PAIS_NACIMIENTO',
       'AFL_MUNICIPIO_NACIMIENTO', 'AFL_NACIONALIDAD',
       'AFL_SEXO_IDENTIFICACION', 'AFL_DISCAPACIDAD', 'TPS_AFL_ID',
       'TPS_PRN_ID', 'TPS_GRP_PBL_ID', 'TPS_NVL_SSB_ID', 'NUMEROFICHASISBEN',
       'TPS_CND_BNF_ID', 'DPR_ID', 'MNC_ID', 'ZNS_ID',
       'AFL_FECHA_AFILIACION_SGSSS', 'AFC_FECHA_INICIO', 'NUMERO CONTRATO',
       'FECHADE INICIO DEL CONTRATO', 'CNT_AFL_TPS_GRP_PBL_ID',
       'CNT_AFL_TPS_PRT_ETN_ID_2', 'TPS_MDL_SBS_ID', 'TPS_EST_AFL_ID',
       'CND_AFL_FECHA_INICIO', 'CND_AFL_FECHA_INICIO', 'GRP_FML_COTIZANTE_ID',
       'PORTABILIDAD', 'COD_IPS_P', 'MTDLG_G_P', 'SUB_SISBEN_IV',
       'MARCASISBENIV+MARCASISBENIII', 'CRUCE_BDEX_RNEC',
       'MARCASISBENIV+MARCASISB